In [1]:
!pip list

Package                  Version
------------------------ ---------
absl-py                  1.4.0
aiofiles                 22.1.0
aiohttp                  3.8.5
aiosignal                1.2.0
aiosqlite                0.18.0
anyio                    3.5.0
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
asttokens                2.0.5
astunparse               1.6.3
async-timeout            4.0.2
attrs                    22.1.0
Babel                    2.11.0
backcall                 0.2.0
beautifulsoup4           4.12.2
bleach                   4.1.0
blinker                  1.4
Bottleneck               1.3.5
brotlipy                 0.7.0
cachetools               4.2.2
category-encoders        2.6.1
certifi                  2022.12.7
cffi                     1.15.1
chardet                  4.0.0
charset-normalizer       2.0.4
click                    8.0.4
colorama                 0.4.6
comm                     0.1.2
contourpy                1.0.5
cryptography           

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict
import json 

In [2]:
# Load and parse JSON data
annotation_path = './train/_annotations.coco.json'

with open(annotation_path, 'r') as json_file:
    data = json.load(json_file)

data

{'info': {'year': '2023',
  'version': '23',
  'description': 'Exported from roboflow.com',
  'contributor': '',
  'url': 'https://public.roboflow.com/object-detection/undefined',
  'date_created': '2023-09-23T08:18:09+00:00'},
 'licenses': [{'id': 1,
   'url': 'https://creativecommons.org/licenses/by/4.0/',
   'name': 'CC BY 4.0'}],
 'categories': [{'id': 0, 'name': 'bills', 'supercategory': 'none'},
  {'id': 1, 'name': '100', 'supercategory': 'bills'},
  {'id': 2, 'name': '1000', 'supercategory': 'bills'},
  {'id': 3, 'name': '20', 'supercategory': 'bills'},
  {'id': 4, 'name': '200', 'supercategory': 'bills'},
  {'id': 5, 'name': '50', 'supercategory': 'bills'},
  {'id': 6, 'name': '500', 'supercategory': 'bills'}],
 'images': [{'id': 0,
   'license': 1,
   'file_name': '50_da9888c614104dbead8eb585aec24302_jpg.rf.617d871d37476e646d6acdfff8ab911f.jpg',
   'height': 640,
   'width': 640,
   'date_captured': '2023-09-23T08:18:09+00:00'},
  {'id': 1,
   'license': 1,
   'file_name': '10

In [3]:
"""
Extract relevant data
Shall not includethe images that doesn't have a banknote on it
"""
annotation_ids = [entry['image_id'] for entry in data['annotations']]

image_file_paths = []
for entry in data['images']:
    if entry['id'] in annotation_ids:
        image_file_paths.append(entry['file_name'])

# image_file_paths

In [4]:
bounding_boxes = []
class_labels = []
for entry in data['annotations']:
    bounding_boxes.append(entry['bbox'])
    class_labels.append(entry['category_id'])

This part uses the Dataframe from pandas to combine the images and their bounding boxes and another dataframe for only the classes (or the predictions to be made)

In [5]:
df_dataset = pd.DataFrame({'image_filename': image_file_paths, 'bounding_boxes': bounding_boxes, 'class_labels': class_labels})

In [7]:
df_dataset

,image_filename,bounding_boxes,class_labels
0,50_da9888c614104dbead8eb585aec24302_jpg.rf.617...,"[0, 97, 134, 447]",5
1,100_eed608e6448e4370965c385ea038096c_jpg.rf.61...,"[518, 0, 122, 445.5]",1
2,50_cbb68bafa85b401d8355684e2d927798_jpg.rf.618...,"[196, 231, 125, 193]",5
3,500_d63c7cd7c4ea4e8d8fd0442ee49bdb9e_jpg.rf.61...,"[331, 0, 307, 176]",6
4,1000_c4e14be1f5574a17890c263b62d7afeb_jpg.rf.6...,"[341, 0, 299, 179]",2
...,...,...,...
15112,500_dfcf4af835344d5794f701e5c9f9dafc_jpg.rf.c0...,"[0, 0, 117, 223]",6
15113,100_c058b1a28f044aecbd611a1b0e4a7470_jpg.rf.bf...,"[5, 4, 158, 441.5]",1
15114,200_3571282a8a9b4ed7a29b3754f11b7eae_jpg.rf.c0...,"[171, 456, 148.5, 183]",4
15115,1000_d253fab7b1d84a008b41eb8f033d2909_jpg.rf.c...,"[256, 5, 121.5, 442.5]",2


In [6]:
X = df_dataset.drop(columns='class_labels')
y = df_dataset['class_labels']

In [9]:
X

,image_filename,bounding_boxes
0,50_da9888c614104dbead8eb585aec24302_jpg.rf.617...,"[0, 97, 134, 447]"
1,100_eed608e6448e4370965c385ea038096c_jpg.rf.61...,"[518, 0, 122, 445.5]"
2,50_cbb68bafa85b401d8355684e2d927798_jpg.rf.618...,"[196, 231, 125, 193]"
3,500_d63c7cd7c4ea4e8d8fd0442ee49bdb9e_jpg.rf.61...,"[331, 0, 307, 176]"
4,1000_c4e14be1f5574a17890c263b62d7afeb_jpg.rf.6...,"[341, 0, 299, 179]"
...,...,...
15112,500_dfcf4af835344d5794f701e5c9f9dafc_jpg.rf.c0...,"[0, 0, 117, 223]"
15113,100_c058b1a28f044aecbd611a1b0e4a7470_jpg.rf.bf...,"[5, 4, 158, 441.5]"
15114,200_3571282a8a9b4ed7a29b3754f11b7eae_jpg.rf.c0...,"[171, 456, 148.5, 183]"
15115,1000_d253fab7b1d84a008b41eb8f033d2909_jpg.rf.c...,"[256, 5, 121.5, 442.5]"


In [7]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [12]:
X_train

,image_filename,bounding_boxes
765,1000_new_70e13adfbcbc40fbbdd57a89636f7201_jpg....,"[338, 0, 297, 179]"
5927,50_c5d27dd2cd5642b69e128a5666a2a124_jpg.rf.049...,"[0, 227, 298, 185]"
14029,500_c96f0fdf87314bd7b3d0603def1daa40_jpg.rf.ae...,"[341, 0, 297, 88]"
6187,20_1d62f6303bda4c568de28bb99406fb85_jpg.rf.08b...,"[0, 191, 63, 449]"
3872,50_f479582edef346909ac4bf31f71f7047_jpg.rf.22d...,"[0, 225, 124, 184]"
...,...,...
5191,20_4268baf0abc547b9bc60db0145bc7a69_jpg.rf.382...,"[270, 191, 49, 449]"
13418,50_c95f69fd402045779ae099156fc3d2f2_jpg.rf.a45...,"[170, 0, 299, 89]"
5390,1000_new_f09cb00135054725a378126b59127449_jpg....,"[0, 456, 640, 95]"
860,100_5ca13652b1274ce3908fbc9a10684ff6_jpg.rf.70...,"[0, 230, 297, 185.5]"


This part are being separated even the bounding box but the problem is that it will not be linked to the images thus will have a problem in annotations

In [16]:
# from sklearn.model_selection import train_test_split

# # Split the data into training, validation, and test sets
# X_train, X_temp, y_train, y_temp, Bbox_train, Bbox_temp = train_test_split(
#     image_file_paths, class_labels, bounding_boxes, test_size=0.3, random_state=42)

# X_val, X_test, y_val, y_test, bbox_val, bbox_test = train_test_split(
#     X_temp, y_temp, Bbox_temp, test_size=0.5, random_state=42)

In [17]:
# X_train

In [18]:
# len(X_test)

In [19]:
# len(X_val)

In [20]:
# Bbox_train

I used the Tensorflow dataset here but didn't continue

In [ ]:
# Data Preprocessing

for items in X_train:
    # Load the image.
    image = tf.io.read_file('train/' + X_train[i])

    # Decode the image.
    image = tf.image.decode_jpeg(image)

    # Normalize the image.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = (image - 0.5) / 0.5

    # Load the annotation bounding box.
    bbox = 

    # Normalize the annotation bounding box.
    bbox = bbox / (224, 224)

    # Expand the image and annotation bounding box to a batch of one.
    image = tf.expand_dims(image, axis=0)
    bbox = tf.expand_dims(bbox, axis=0)

    # Print the normalized image and annotation bounding box.
    print(image)
    print(bbox)

TypeError: unsupported operand type(s) for /: 'list' and 'tuple'

In [22]:
import tensorflow as tf

X = tf.data.Dataset.from_tensor_slices((image_file_paths, bounding_boxes))
y = tf.data.Dataset.from_tensor_slices(class_labels)
# for element in dataset:
    # print(element)

In [25]:
y

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [27]:
for (image_path, bbox) in X.take(5):  # Take the first 5 elements for visualization
    print("Image Path:", image_path)
    print("Bounding Box:", bbox)
    print("\n")

Image Path: tf.Tensor(b'50_da9888c614104dbead8eb585aec24302_jpg.rf.617d871d37476e646d6acdfff8ab911f.jpg', shape=(), dtype=string)
Bounding Box: tf.Tensor([  0.  97. 134. 447.], shape=(4,), dtype=float32)


Image Path: tf.Tensor(b'100_eed608e6448e4370965c385ea038096c_jpg.rf.61a42761472f9bd806896bd88069af2d.jpg', shape=(), dtype=string)
Bounding Box: tf.Tensor([518.    0.  122.  445.5], shape=(4,), dtype=float32)


Image Path: tf.Tensor(b'50_cbb68bafa85b401d8355684e2d927798_jpg.rf.618388d8428725935c62c4dfbc6d9ef6.jpg', shape=(), dtype=string)
Bounding Box: tf.Tensor([196. 231. 125. 193.], shape=(4,), dtype=float32)


Image Path: tf.Tensor(b'500_d63c7cd7c4ea4e8d8fd0442ee49bdb9e_jpg.rf.61845f728667466eb727eb2bab835f63.jpg', shape=(), dtype=string)
Bounding Box: tf.Tensor([331.   0. 307. 176.], shape=(4,), dtype=float32)


Image Path: tf.Tensor(b'1000_c4e14be1f5574a17890c263b62d7afeb_jpg.rf.619f14d0f375dc3c3292a94796037e65.jpg', shape=(), dtype=string)
Bounding Box: tf.Tensor([341.   0. 299

In [28]:
for label in y.take(5):  # Take the first 5 elements for visualization
    print("Label:", label)
    print("\n")

Label: tf.Tensor(5, shape=(), dtype=int32)


Label: tf.Tensor(1, shape=(), dtype=int32)


Label: tf.Tensor(5, shape=(), dtype=int32)


Label: tf.Tensor(6, shape=(), dtype=int32)


Label: tf.Tensor(2, shape=(), dtype=int32)




In [11]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((image_file_paths, bounding_boxes, class_labels))

Normalization of the images

In [8]:
import cv2  # OpenCV for image loading and manipulation
import tensorflow as tf

# Define a function to load and normalize images
def load_and_normalize_image(filename):
    # Load the image using OpenCV (you can use other libraries like PIL as well)
    image = cv2.imread('train/' + filename)

    # Check if the image is loaded successfully
    if image is None:
        raise FileNotFoundError(f"Image not found: {filename}")
    
    # Resize the images
    image = tf.image.resize(image, (300, 300))
    
    # Normalize pixel values to the range [0, 1]
    normalized_image = image / 255.0

    return normalized_image

# Apply the normalization function to the 'filename' column
X_train['image_filename'] = X_train['image_filename'].apply(load_and_normalize_image)

In [ ]:
X_train